In [2]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re
import random


class Propiedad:
    """Representa una propiedad inmobiliaria con sus atributos básicos."""

    def __init__(self, precio, tamano, habitaciones, link):
        self.precio = self._limpiar(precio)
        self.tamano = self._limpiar(tamano)
        self.habitaciones = self._limpiar(habitaciones)
        self.link = link

    @staticmethod
    def _limpiar(texto):
        """Extrae números enteros de un texto (ej: '120 m²' -> 120)."""
        if not texto or not isinstance(texto, str):
            return None
        match = re.search(r"\d+", texto.replace(".", ""))
        return int(match.group(0)) if match else None

    def to_dict(self):
        """Convierte el objeto en un diccionario JSON serializable."""
        return {
            "precio": self.precio,
            "tamano": self.tamano,
            "habitaciones": self.habitaciones,
            "link": self.link
        }


class CiudadScraper:
    """Se encarga de raspar las propiedades de una ciudad específica."""

    BASE_URL = "https://inmuebles.mercadolibre.com.uy/casas/venta/"
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/91.0.4472.124 Safari/537.36"
    }

    def __init__(self, nombre, url_fragmento):
        self.nombre = nombre
        self.url = f"{self.BASE_URL}{url_fragmento}/"
        self.propiedades = []

    def extraer(self, limite=10):
        """Raspa hasta 'limite' propiedades de la ciudad."""
        try:
            response = requests.get(self.url, headers=self.HEADERS, timeout=15)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            items = soup.select("li.ui-search-layout__item", limit=limite)

            for item in items:
                moneda = item.select_one("span.andes-money-amount__currency-symbol")
                if not moneda or "U$S" not in moneda.text:
                    continue

                precio = item.select_one("span.andes-money-amount__fraction").text.strip()
                link = item.select_one("a.ui-search-link")["href"]

                # Atributos adicionales
                tamano, habitaciones = "No disponible", "No disponible"
                atributos = item.select("div.ui-search-card__attributes-container p.ui-search-card__attribute")
                for attr in atributos:
                    txt = attr.text.strip()
                    if "m²" in txt:
                        tamano = txt
                    elif "dormitorio" in txt:
                        habitaciones = txt

                self.propiedades.append(Propiedad(precio, tamano, habitaciones, link))

        except requests.exceptions.RequestException as e:
            print(f"❌ Error al conectar con {self.url}: {e}")

        # Pausa aleatoria para evitar bloqueo
        time.sleep(random.uniform(1.5, 3.0))

    def to_dict(self):
        """Convierte la ciudad y sus propiedades a dict."""
        return {
            "nombre": self.nombre,
            "propiedades": [p.to_dict() for p in self.propiedades]
        }


class ScraperInmuebles:
    """Coordina el scraping de múltiples ciudades."""

    def __init__(self, ciudades):
        self.ciudades = [CiudadScraper(nombre, url) for nombre, url in ciudades.items()]

    def ejecutar(self):
        resultado = {"ciudades": []}
        for ciudad in self.ciudades:
            print(f"--- 🏙️ Scrapeando {ciudad.nombre} ---")
            ciudad.extraer()
            resultado["ciudades"].append(ciudad.to_dict())
        return resultado


if __name__ == "__main__":
    ciudades_uruguay = {
        "Montevideo": "montevideo",
        "Punta del Este": "maldonado/punta-del-este",
        "Canelones": "canelones"
    }

    scraper = ScraperInmuebles(ciudades_uruguay)
    datos = scraper.ejecutar()

    # Guardar en JSON
    with open("propiedades.json", "w", encoding="utf-8") as f:
        json.dump(datos, f, ensure_ascii=False, indent=4)

print("\n✅ ¡Proceso completado! Datos guardados en 'propiedades.json'")


--- 🏙️ Scrapeando Montevideo ---
--- 🏙️ Scrapeando Punta del Este ---
--- 🏙️ Scrapeando Canelones ---

✅ ¡Proceso completado! Datos guardados en 'propiedades.json'
